In [32]:
import requests
import json
from google import genai
from google.genai import types as gt
import os

REDMINE_API_KEY = "00b97c8f09f4e8498c720f0043b79930ef9c2823"
REDMINE_PROJECT_ID = "testproject-redmine"

url = "http://localhost:3000/issues.json?project_id=testproject-redmine"
headers = {
    "X-Redmine-API-Key": "00b97c8f09f4e8498c720f0043b79930ef9c2823"
}

In [33]:
# 1. 拉取 Redmine 工单
def redmine_get_issues(api_key, project_id, url_base="http://localhost:3000"):
    url = f"{url_base}/issues.json?project_id={project_id}"
    headers = {"X-Redmine-API-Key": api_key}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["issues"]

# 2. 格式化工单列表为字符串
def format_redmine_issues_to_str(issues):
    lines = []
    for i in issues:
        subject = i.get("subject", "无标题")
        status = i.get("status", {}).get("name", "未知状态")
        priority = i.get("priority", {}).get("name", "无优先级")
        assignee = (i.get("assigned_to") or {}).get("name", "未分配")
        author = (i.get("author") or {}).get("name", "未知作者")
        start_date = i.get("start_date", "无开始日期")
        due_date = i.get("due_date", "无截止日期")
        created_on = i.get("created_on", "未知创建时间")
        updated_on = i.get("updated_on", "未知更新时间")
        desc = i.get("description", "无描述")
        lines.append(
            f"{subject} | 状态: {status} | 优先级: {priority} | 负责人: {assignee} | 作者: {author} | 开始: {start_date} | 截止: {due_date} | 创建: {created_on} | 更新: {updated_on} | 描述: {desc[:20]}..."
        )
    return "\n".join(lines)

In [34]:
# 3. 获取 Redmine 工单并格式化
issues = redmine_get_issues(REDMINE_API_KEY, REDMINE_PROJECT_ID)
issues_str = format_redmine_issues_to_str(issues)
print("Formatted Redmine issues:")
print(issues_str)

Formatted Redmine issues:
develop new feature 2 | 状态: New | 优先级: High | 负责人: dun dun | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: 2025-10-01 | 创建: 2025-09-30T07:45:43Z | 更新: 2025-09-30T07:45:43Z | 描述: develop new feature ...
fix urgent bug 1 | 状态: New | 优先级: Urgent | 负责人: Redmine Admin | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: None | 创建: 2025-09-30T07:39:53Z | 更新: 2025-09-30T07:39:53Z | 描述: fix urgent bug 1...
develop new feature 1 | 状态: New | 优先级: Normal | 负责人: Redmine Admin | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: 2025-10-08 | 创建: 2025-09-30T07:38:54Z | 更新: 2025-09-30T07:38:54Z | 描述: develop new feature ...
fix new bug 1 | 状态: New | 优先级: Normal | 负责人: Redmine Admin | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: 2025-10-03 | 创建: 2025-09-30T07:38:12Z | 更新: 2025-09-30T11:16:04Z | 描述: fix new bug 1...


In [ ]:
# 5. 调用 Gemini 做 Redmine 工单分析
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

prompt = (
    "你是资深项目管理顾问，请根据以下 Redmine 工单列表，输出结构化的分析建议。\n"
    "请严格按照如下格式返回：\n"
    "项目建议：...\n"
    "排期管理建议：...\n"
    "人员分配建议：...\n"
    "要求：\n"
    "- 项目建议：针对整体项目进展、风险、优先级等提出建议。\n"
    "- 排期管理建议：对任务截止时间、进度延误、合理性等给出建议。\n"
    "- 人员分配建议：对当前人员分配合理性、负载、改进方向等给出建议。\n"
    "\n以下是 Redmine 工单列表：\n\n"
    + issues_str
)

print("\n=== Prompt to Gemini ===")
print(prompt)

resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=gt.GenerateContentConfig(temperature=0.3)
)

# 6. 获取 Gemini 分析结果
gemini_response = resp.text
print("=== AI 分析结果 ===")
print(gemini_response)


=== Prompt to Gemini ===
你是资深项目管理顾问，请根据以下 Redmine 工单列表，输出结构化的分析建议。
请严格按照如下格式返回：
项目建议：...
排期管理建议：...
人员分配建议：...
要求：
- 项目建议：针对整体项目进展、风险、优先级等提出建议。
- 排期管理建议：对任务截止时间、进度延误、合理性等给出建议。
- 人员分配建议：对当前人员分配合理性、负载、改进方向等给出建议。

以下是 Redmine 工单列表：

develop new feature 2 | 状态: New | 优先级: High | 负责人: dun dun | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: 2025-10-01 | 创建: 2025-09-30T07:45:43Z | 更新: 2025-09-30T07:45:43Z | 描述: develop new feature ...
fix urgent bug 1 | 状态: New | 优先级: Urgent | 负责人: Redmine Admin | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: None | 创建: 2025-09-30T07:39:53Z | 更新: 2025-09-30T07:39:53Z | 描述: fix urgent bug 1...
develop new feature 1 | 状态: New | 优先级: Normal | 负责人: Redmine Admin | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: 2025-10-08 | 创建: 2025-09-30T07:38:54Z | 更新: 2025-09-30T07:38:54Z | 描述: develop new feature ...
fix new bug 1 | 状态: New | 优先级: Normal | 负责人: Redmine Admin | 作者: Redmine Admin | 开始: 2025-09-30 | 截止: 2025-10-03 | 创建: 2025-09-30T07:38:12Z | 更新: 2025-09-30T11:16:04Z | 描述: fix 

In [36]:
# 7. （可选）将 Gemini 分析结果写入本地文件
with open("redmine_gemini_analysis.txt", "w") as f:
    f.write(gemini_response)

In [37]:
issue_id = 1  # 替换为你的工单ID
api_key = "00b97c8f09f4e8498c720f0043b79930ef9c2823"
redmine_url = "http://localhost:3000"
comment = "这里是AI分析建议内容" + gemini_response

url = f"{redmine_url}/issues/{issue_id}.json"
headers = {
    "X-Redmine-API-Key": api_key,
    "Content-Type": "application/json; charset=utf-8"
}
data = {
    "issue": {
        "notes": comment
    }
}

response = requests.put(
    url,
    data=json.dumps(data, ensure_ascii=False).encode("utf-8"),
    headers=headers
)
print(response.status_code)
if response.headers.get("Content-Type", "").startswith("application/json"):
    print(response.json())
else:
    print("不是JSON，原始内容如上")

204
不是JSON，原始内容如上
